In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install --upgrade pip
!pip install --upgrade transformers datasets[audio] accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 24.0
    Uninstalling pip-24.0:
      Successfully uninstalled pip-24.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 82.6 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.34.2
    Uninstalling accelerate-0.34.2:
      Successfully uninstalled accelerate-0.34.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.45.1
    Uninstalling transformers-4.45.1:
      Successfully uninstalled transformers-4.45.1


In [2]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import librosa

# Set up device and data type
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

# Load the model
model_id = "openai/whisper-large-v3-turbo"
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

# Load the processor
processor = AutoProcessor.from_pretrained(model_id)

# Create the pipeline
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    chunk_length_s=30,  # Process 30-second chunks
    batch_size=8,  # Adjust based on your GPU memory
    torch_dtype=torch_dtype,
    device=device,
)

# Load your audio file
audio_path = "/kaggle/input/audio-file/output_audio.wav"  # Replace with your audio file path
audio, sr = librosa.load(audio_path, sr=16000)

model.safetensors:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.77k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

In [4]:

audio_path = "/kaggle/input/lecture/lecture_audio.wav"  
audio, sr = librosa.load(audio_path, sr=16000)
result = pipe(audio, generate_kwargs={"language": "english"})  

print(result["text"])


with open("transcription.txt", "w") as f:
    f.write(result["text"])

 Screen also visible now? Yes ma'am. Thank you. Let us start with the next part that is statistical inference. We will learn upon test of significance and from here we will learn about three tests that is chi-square test, t-test and f-test. But before that we have to learn about what is this test of significance. What do we mean by statistical inference? What is the importance of statistical inference in design of experiments? And then we will come to the type of hypothesis. And then we will start upon the different tests we have set. So statistical inference is to deduce, right? Is to infer whether the decision you are making upon a hypothesis is correct or not. Whenever you have your design of experiments, whenever you do experimentation at the end of your design, then you have your framework which has been built using different components, now you have different settings as well. And now for for these settings you will decide to test whether whatever assumptions you have started wit

In [41]:
!pip install optimum

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [48]:
!pip install --upgrade transformers
!pip install --upgrade auto-gptq
!pip install --upgrade optimum


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [49]:
from transformers import AutoTokenizer
from auto_gptq import AutoGPTQForCausalLM
import torch

def organize_lecture_content(input_file, output_file, max_length=2048):
    """
    Organizes a lecture transcript into a structured format using a quantized Llama model.

    Parameters:
    - input_file (str): Path to the input text file containing the lecture transcript.
    - output_file (str): Path to the output text file where the organized content will be saved.
    - max_length (int): Maximum number of new tokens to generate in the organized content.

    Returns:
    - organized_content (str): The organized lecture content.
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    # Define the quantized model ID
    model_id = "neuralmagic/Meta-Llama-3-8B-Instruct-quantized.w8a16"
    
    try:
        # Load tokenizer
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        
        # Load quantized model using from_quantized
        model = AutoGPTQForCausalLM.from_quantized(
            model_id,
            use_safetensors=True,          # Use safetensors if available
            trust_remote_code=True,        # Only set to True if you trust the model's source
            device_map="auto",             # Automatically map model to available devices
        )
        
        # Ensure the model is in evaluation mode
        model.eval()
        
    except Exception as e:
        print(f"Error loading model or tokenizer: {e}")
        return None
    
    try:
        # Read the input lecture transcript
        with open(input_file, 'r', encoding='utf-8') as file:
            text = file.read()
    except Exception as e:
        print(f"Error reading input file: {e}")
        return None
    
    # Define the prompt for organizing the lecture content
    prompt = f"""
    Organize the following lecture transcript into a structured format with these components:
    1. Introduction: Briefly introduce the main topic.
    2. Key Concepts: List and explain the main ideas or theories discussed.
    3. Detailed Explanations: Provide in-depth explanations of each key concept.
    4. Real-Life Examples: For each concept, provide a practical, real-world example.
    5. Important Definitions: List crucial terms or definitions introduced.
    6. Discussion Points: Highlight questions or topics for further discussion.
    7. Summary: Concisely summarize the main takeaways.
    8. Further Reading: Suggest resources for additional study.

    Ensure each section is clearly labeled and use bullet points where appropriate. 
    Create a comprehensive, well-organized document that captures the essence of the lecture 
    and makes it easy for students to review and understand the material.

    Lecture transcript to organize:
    {text}

    Organized lecture content:
    """
    
    try:
        # Tokenize the prompt
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        
        # Generate the organized content
        with torch.no_grad():
            output = model.generate(
                **inputs,
                max_new_tokens=max_length,
                num_return_sequences=1,
                temperature=0.7,
                top_p=0.95,
                eos_token_id=tokenizer.eos_token_id,  # Ensure proper end of generation
            )
        
        # Decode the output
        organized_content = tokenizer.decode(output[0], skip_special_tokens=True)
        
        # Remove the original prompt from the output
        organized_content = organized_content.replace(prompt, "").strip()
        
    except Exception as e:
        print(f"Error during model inference: {e}")
        return None
    
    try:
        # Save the organized content to the output file
        with open(output_file, 'w', encoding='utf-8') as file:
            file.write(organized_content)
    except Exception as e:
        print(f"Error writing to output file: {e}")
        return None
    
    print("Lecture content organized successfully.")
    return organized_content

# Example usage
if __name__ == "__main__":
    input_path = "/kaggle/input/transcription-1/transcription (3).txt"    # Replace with your input file path
    output_path = "organized_lecture.txt"   # Replace with your desired output file path
    organized = organize_lecture_content(input_path, output_path)
    if organized:
        print("Organized Content:\n")
        print(organized)


WARNING - Exllamav2 kernel is not installed, reset disable_exllamav2 to True. This may because you installed auto_gptq using a pre-build wheel on Windows, in which exllama_kernels are not compiled. To use exllama_kernels to further speedup inference, you can re-install auto_gptq from source.
WARNING - CUDA kernels for auto_gptq are not installed, this will result in very slow inference speed. This may because:
1. You disabled CUDA extensions compilation by setting BUILD_CUDA_EXT=0 when install auto_gptq from source.
2. You are using pytorch without CUDA support.
3. CUDA and nvcc are not installed in your device.
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
WARNING - ignoring unknown parameter in config.json: batch_size.
WARNING - ignoring unknown para

model.safetensors.index.json:   0%|          | 0.00/78.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.10G [00:00<?, ?B/s]

INFO - The layer lm_head is not quantized.


  0%|          | 0/545 [00:00<?, ?w/s]

  0%|          | 0/418 [00:00<?, ?w/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (8192). Depending on the model, you may observe exceptions, performance de

Lecture content organized successfully.
Organized Content:

**Introduction**
     * Statistical inference
     * Test of significance
     * Hypothesis testing

     **Key Concepts**
     * Null hypothesis (H0)
     * Alternate hypothesis (H1)
     * Type 1 error (alpha)
     * Type 2 error (beta)
     * Significance level (alpha)
     * One-tailed test
     * Two-tailed test

     **Detailed Explanations**
     * Null hypothesis: a quantitative statement about a population
     * Alternate hypothesis: specifies values that the researcher believes to hold true
     * Type 1 error: rejecting a true null hypothesis
     * Type 2 error: failing to reject a false null hypothesis
     * Significance level: the probability of rejecting a true null hypothesis
     * One-tailed test: used when the estimated value deviates from the reference value in one direction only
     * Two-tailed test: used when the estimated value deviates from the reference value in both directions

     **Real-Life Ex

In [44]:
!pip install --upgrade auto-gptq

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [23]:
!pip install optimum

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.6/422.6 kB 8.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.7 MB/s eta 0:00:00


In [36]:
input_file = "/kaggle/input/transcription-1/transcription (3).txt"

output_file = "organized_lecture_content.txt"
organized_lecture = organize_lecture_content(input_file, output_file)
print("Organized lecture content has been generated and saved to", output_file)
print("\nOrganized Content (first 500 characters):")
print(organized_lecture[:500] + "...")

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

PackageNotFoundError: No package metadata was found for auto-gptq

In [33]:
!pip install --upgrade transformers torch bitsandbytes

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [31]:
!pip install -U bitsandbytes

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
